In [1]:
%pip install pandas scikit-learn openpyxl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [3]:
file_path = 'Jalen Bronson Regular Season Totals (1).xlsx'
df = pd.read_excel(file_path)

In [4]:
columns = [
    'Field Goals Made Per Season', 'Field Goals Attempted Per Season',
    '3-Pointers Made Per Season', '3-Pointers Attempted Per Season',
    'Free Throws Made Per Season', 'Free Throws Attempted Per Season',
    'Offensive Rebounds Per Season', 'Defensive Rebounds Per Season',
    'Assists Per Season', 'Blocks Per Season', 'Steals Per Season',
    'Fouls Per Season', 'Turnovers Per Season'
]

In [5]:
# Filters out rows with 'Career Totals' or NaN in 'SEASON'
df_filtered = df[df['SEASON'].notna() & (df['SEASON'] != 'Career Totals')]

In [6]:
# Coverts 'SEASON' to numerical values for modeling
df_filtered['SEASON'] = pd.to_numeric(df_filtered['SEASON'].str[:4])

/var/folders/j8/641rqqd96ylgrx0_5bw4j3rm0000gq/T/ipykernel_1683/75866017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['SEASON'] = pd.to_numeric(df_filtered['SEASON'].str[:4])


In [7]:
predictions = {}

for column in columns:
    # Prepares the data
    X = df_filtered[['SEASON']].values
    y = df_filtered[column].values

    # Removes missing values
    mask = ~np.isnan(y)
    X, y = X[mask], y[mask]

    # Fits the linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict the value for the 2024-2025 season (2024 as the SEASON value)
    prediction = model.predict(np.array([[2024]]))

    # Rounds and stores the prediction
    predictions[column] = round(prediction[0])

In [8]:
print("Predictions for the 2024-2025 NBA season:")
print(predictions)

Predictions for the 2024-2025 NBA season:
{'Field Goals Made Per Season': 845, 'Field Goals Attempted Per Season': 1727, '3-Pointers Made Per Season': 207, '3-Pointers Attempted Per Season': 505, 'Free Throws Made Per Season': 451, 'Free Throws Attempted Per Season': 531, 'Offensive Rebounds Per Season': 49, 'Defensive Rebounds Per Season': 272, 'Assists Per Season': 557, 'Blocks Per Season': 15, 'Steals Per Season': 80, 'Fouls Per Season': 162, 'Turnovers Per Season': 191}


In [9]:
# Prepares data for total points predictions
X = df_filtered[columns].values
y = df_filtered['Points Per Season'].values

In [10]:
# Removes missing values
mask = ~np.isnan(y) & ~np.isnan(X).any(axis=1)
X, y = X[mask], y[mask]

In [11]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [12]:
predicted_features = np.array([list(predictions.values())])

In [13]:
total_points_prediction = model.predict(predicted_features)

In [14]:
predictions['Points Per Season'] = round(total_points_prediction[0])

In [15]:
print("Updated predictions including Points Per Season for 2024-2025 NBA season:")
print(predictions)

Updated predictions including Points Per Season for 2024-2025 NBA season:
{'Field Goals Made Per Season': 845, 'Field Goals Attempted Per Season': 1727, '3-Pointers Made Per Season': 207, '3-Pointers Attempted Per Season': 505, 'Free Throws Made Per Season': 451, 'Free Throws Attempted Per Season': 531, 'Offensive Rebounds Per Season': 49, 'Defensive Rebounds Per Season': 272, 'Assists Per Season': 557, 'Blocks Per Season': 15, 'Steals Per Season': 80, 'Fouls Per Season': 162, 'Turnovers Per Season': 191, 'Points Per Season': 2348}


In [16]:
# New row for 2024-25 season
new_row = {'SEASON': '2024-25', 'TEAM': 'NY'}
for column, value in predictions.items():
    new_row[column] = value

In [17]:
# Appends new row to DataFrame
df = df.append(new_row, ignore_index=True)

In [18]:
# Calculate Career Totals
career_totals = df[columns].sum()
career_totals['SEASON'] = 'Career Totals'
career_totals['TEAM'] = np.nan

In [19]:
df = df.append(career_totals, ignore_index=True)

In [20]:
print("Updated DataFrame with predictions for the 2024-2025 season and Career Totals:")
print(df)

Updated DataFrame with predictions for the 2024-2025 season and Career Totals:
           SEASON TEAM  Field Goals Made Per Season  \
0         2018-19  DAL                        264.0   
1         2019-20  DAL                        181.0   
2         2020-21  DAL                        328.0   
3         2021-22  DAL                        506.0   
4         2022-23   NY                        587.0   
5         2023-24   NY                        790.0   
6         2024-25   NY                          NaN   
7             NaN  NaN                          NaN   
8   Career Totals  NaN                       2656.0   
9         2024-25   NY                        845.0   
10        2024-25   NY                        845.0   
11        2024-25   NY                        845.0   
12        2024-25   NY                        845.0   
13  Career Totals  NaN                       8692.0   

    Field Goals Attempted Per Season  3-Pointers Made Per Season  \
0                          

In [22]:
df.to_excel(file_path, index=False)